<div id='top'>包括以下操作：</div>
<li><a href='#1'>整合数据集，并划分数据集（jpg、xml）</a></li>
<li><a href='#2'>xml文件转为txt文件</a></li>
<li><a href='#3'></a></li>
<li><a href='#4'></a></li>
<li><a href='#5'></a></li>
<li><a href='#6'></a></li>

### <a id='1'>整合数据集，并划分数据集（jpg、xml）</a>  

<a href='#top'>▲ Top</a>

In [ ]:
import os
import random
import glob
import json
import shutil
from xml.etree import ElementTree as ET


def json_split_train_and_val(train_percent, val_percent, annotation_path, train_and_val_txt_path):
    json_annotation_path = annotation_path
    total_json = os.listdir(json_annotation_path)
    random.shuffle(total_json)

    num_json = len(total_json)
    num_train = int(train_percent * num_json)
    num_val = int(val_percent * num_json)
    train_list = total_json[:num_train]
    val_list = total_json[num_train:]

    for i in train_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "train")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.move(src_path, dst_path)

    for i in val_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "val")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.move(src_path, dst_path)


def xml_split_train_and_val(train_percent, val_percent, annotation_path, train_and_val_txt_path):
    xml_annotation_path = annotation_path
    total_xml = os.listdir(xml_annotation_path)
    random.shuffle(total_xml)

    num_xml = len(total_xml)
    num_train = int(train_percent * num_xml)
    num_val = int(val_percent * num_xml)
    train_list = total_xml[:num_train]
    val_list = total_xml[num_train:]

    for i in train_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "train")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.copy(src_path, dst_path)

    for i in val_list:
        name = i[:-4] + ".jpg"
        src_path = os.path.join(train_and_val_txt_path, name)
        dst_dir = os.path.join(train_and_val_txt_path.replace("JPEGImages", "images"), "val")
        if not os.path.exists(dst_dir):
            os.makedirs(dst_dir)

        dst_path = os.path.join(dst_dir, name)

        shutil.copy(src_path, dst_path)


def move_jpg_xml(total_jpg_dir, annotation_path, JPEGImages_path,images_path,labels_path, not_concat=True,delete_file=False):
    if not_concat:
        # 检查jpg文件夹是否存在且非空
        if os.path.exists(JPEGImages_path) and any(os.scandir(JPEGImages_path)):
            # 删除文件夹及其内容
            shutil.rmtree(JPEGImages_path)
            os.makedirs(JPEGImages_path)

        # 检查xml文件夹是否存在且非空
        if os.path.exists(annotation_path) and any(os.scandir(annotation_path)):
            # 删除文件夹及其内容
            shutil.rmtree(annotation_path)
            os.makedirs(annotation_path)

        # 检查images文件夹是否存在且非空
        if os.path.exists(images_path) and any(os.scandir(images_path)):
            # 删除文件夹及其内容
            shutil.rmtree(images_path)
            os.makedirs(images_path)

        # 检查labels文件夹是否存在且非空
        if os.path.exists(labels_path) and any(os.scandir(labels_path)):
            # 删除文件夹及其内容
            shutil.rmtree(labels_path)
            os.makedirs(labels_path)

    jpg_list = []
    xml_List = []
    for root,dirs,files in os.walk(total_jpg_dir):
        for img_i in files:
            if any(img_i.lower().endswith(ext) for ext in ['.jpg', '.jpeg', '.png']): # 对于所有为图片的文件，不管是哪个层级的

                xml_i = img_i.replace("jpg", "xml")
                img_i_path = os.path.join(root, img_i).replace('\\', '/')
                xml_i_path = os.path.join(root, xml_i).replace('\\', '/')
                print('img_i_path == ', img_i_path)
                print('xml_i_path == ', xml_i_path)
                if not os.path.exists(xml_i_path):
                    os.remove(img_i_path) # 里面是路径
                    # continue # 跳出本次循环，不会继续执行之后的代码
                else:
                    xml_List.append(xml_i_path)

    for xml_i_path in xml_List:
        xml_data = ET.parse(xml_i_path)
        img_i_path = xml_i_path.replace("xml", "jpg")
        root = xml_data.getroot()

        object_state = root.findall("object")
        if len(object_state) != 0:
            # 移动jpg
            jpg_name = os.path.basename(img_i_path) # 返回path最后的文件名
            xml_name = os.path.basename(xml_i_path)

            src_img_path = img_i_path
            dst_img_path = os.path.join(JPEGImages_path, jpg_name).replace('\\', '/')
            shutil.copy(src_img_path, dst_img_path)

            # 移动xml
            src_xml_path = xml_i_path
            dst_xml_path = os.path.join(annotation_path, xml_name).replace('\\', '/')
            shutil.copy(src_xml_path, dst_xml_path)
        else:
            # 如果没有标注框，就对图片和xml进行删除
            os.remove(img_i_path)
            os.remove(xml_i_path)



if __name__ == "__main__":
    train_percent = 0.8
    val_percent = 0.2
    # 自己现在目录里创建 annotation_path、JPEGImages_path、images_path、labels_path 定义的文件夹
    annotation_path = r"Annotations"
    JPEGImages_path = r"JPEGImages"
    images_path = r"images"
    labels_path = r"labels"
    # total_jpg_dir = r"testData/**/*.jpg" # testData\\D1-C3\\id=73_35_Scort=20.2552.jpg
    # total_jpg_dir = r"testData/**/**/*.jpg" # testData\\test2\\D1-C2\\id=73_35_Scort=20.2552.jpg
    total_jpg_dir = r"testData"
    # total_jpg_dir = r"/gpfs/users/aiAnalyze/jilei/chicken_blood/立华/未验证/20240701以后不需要胡一泓验证直接训练，因此这些数据不再放进已验证/20240718立华/20240614-F511-512已看已改/**/*.jpg"
    # total_jpg_dir = r"E:\DataSets\chicken_blood\立华\未验证\20240701以后不需要胡一泓验证直接训练，因此这些数据不再放进已验证\20240724_temp/*.jpg"

    move_jpg_xml(total_jpg_dir, annotation_path, JPEGImages_path,images_path,labels_path, not_concat=True,delete_file=False) # 第一次为 True，后续为 False，这样去合并文件
    # 功能：训练和验证数据划分，可等待move_jpg_xml函数汇总完数据后运行 # 全部数据汇总完毕后再运行
    # xml_split_train_and_val(train_percent, val_percent, annotation_path, JPEGImages_path)


### <a id='2'>xml文件转为txt文件</a>  

<a href='#top'>▲ Top</a>

里面记录的是 标签 以及 标注框的坐标